<a href="https://colab.research.google.com/github/Manojlamdade/Manojlamdade/blob/main/Rick_and_Morty_chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import  pandas as pd
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/")

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1949: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
pip install transformers datasets pandas torch

In [ ]:
import pandas as pd
import re
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

# Step 1: Load and Clean the Dataset
def load_and_clean_data(file_path):
    df = pd.read_csv(file_path)

    # Remove non-conversational lines (e.g., sound effects)
    def is_conversational(line):
        if pd.isna(line):
            return False
        # Remove lines that are mostly screams or sound effects
        if re.match(r'^(A{2,}|O{2,}|H{2,}|!{2,}|\s*)+$', line, re.IGNORECASE):
            return False
        return True

    df = df[df['line'].apply(is_conversational)]

    # Normalize text (optional: lowercase, remove excessive punctuation)
    df['line'] = df['line'].str.strip()
    return df

# Step 2: Group Conversations
def group_conversations(df):
    conversations = []
    current_convo = []
    current_episode = None

    for _, row in df.iterrows():
        episode = (row['season no.'], row['episode no.'])

        # Start a new conversation if episode changes
        if current_episode != episode and current_convo:
            conversations.append(current_convo)
            current_convo = []

        current_convo.append((row['name'], row['line']))
        current_episode = episode

    # Append the last conversation
    if current_convo:
        conversations.append(current_convo)

    return conversations

# Step 3: Format for DialoGPT
def format_conversations(conversations):
    formatted_convos = []
    for convo in conversations:
        formatted = ""
        for speaker, line in convo:
            formatted += f"{speaker}: {line} <|endoftext|>\n"
        formatted_convos.append(formatted.strip())
    return formatted_convos

# Step 4: Create Hugging Face Dataset
def create_hf_dataset(formatted_convos):
    return Dataset.from_dict({"text": formatted_convos})

# Step 5: Tokenize Data
def tokenize_data(dataset, tokenizer, max_length=512):
    def tokenize_function(examples):
        encodings = tokenizer(
            examples['text'],
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        # Labels are the same as input_ids for causal language modeling
        encodings['labels'] = encodings['input_ids'].clone()
        return encodings

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

# Step 6: Fine-Tune the Model
def fine_tune_model(tokenized_dataset, model_name="microsoft/DialoGPT-small", output_dir=r"/content/drive/MyDrive/Colab Notebooks/dialogpt_rick_morty"):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Split dataset
    train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
    train_dataset = train_test_split['train']
    eval_dataset = train_test_split['test']

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=500,
        load_best_model_at_end=True,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Train the model
    trainer.train()

    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    23
    return model, tokenizer

# Main Pipeline
def main(file_path=r"/content/drive/MyDrive/Colab Notebooks/Rick and Morty.csv"):
    # Preprocessing
    df = load_and_clean_data(file_path)
    conversations = group_conversations(df)
    formatted_convos = format_conversations(conversations)
    dataset = create_hf_dataset(formatted_convos)

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
    tokenizer.pad_token = tokenizer.eos_token

    # Tokenize
    tokenized_dataset = tokenize_data(dataset, tokenizer)

    # Fine-tune
    model, tokenizer = fine_tune_model(tokenized_dataset)

    print(f"Model and tokenizer saved to ./dialogpt_rick_morty")
    if _name_ == "_main_":main()

In [ ]:
all_rick = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Rick and Morty.csv')
all_rick.head(10)

,index,season no.,episode no.,episode name,name,line
0,0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."
5,5,1,1,Pilot,Morty,Ow! Ow! You're tugging me too hard!
6,6,1,1,Pilot,Rick,"We gotta go, gotta get outta here, come on. Go..."
7,7,1,1,Pilot,Rick,"What do you think of this... flying vehicle, M..."
8,8,1,1,Pilot,Morty,"Yeah, Rick... I-it's great. Is this the surprise?"
9,9,1,1,Pilot,Rick,Morty. I had to... I had to do it. I had— I ha...


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from google.colab import drive, files
import random

# Preprocess text, retaining some punctuation
def preprocess_text(text):
    text = text.lower().strip()
    text = re.sub(r'[^\w\s!?]', '', text)  # Keep ! and ?
    return ' '.join(text.split())  # Normalize whitespace

# Mount Google Drive or upload file
def load_data(file_path="/content/drive/MyDrive/Colab Notebooks/Rick and Morty.csv"):
    try:
        if not file_path.startswith('/content/drive') or not pd.io.common.file_exists(file_path):
            drive.mount('/content/drive', force_remount=True)
            if not pd.io.common.file_exists(file_path):
                print("Dataset not found. Please upload 'Rick and morty.csv'.")
                uploaded = files.upload()
                if 'Rick and morty.csv' not in uploaded:
                    raise FileNotFoundError("Upload failed. Ensure file is named 'Rick and morty.csv'.")
                pd.read_csv('Rick and morty.csv').to_csv(file_path, index=False)
        return pd.read_csv(file_path)
    except Exception as e:
        raise FileNotFoundError(f"Error loading dataset: {str(e)}")

# Get response based on similarity
def get_response(user_input, data, vectorizer, tfidf_matrix):
    if not user_input.strip():
        return "Rick: burp Say somethin’ meaningful, Morty!"
    processed_input = preprocess_text(user_input)
    input_vector = vectorizer.transform([processed_input])
    similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()

    # Get top 3 most similar indices
    top_indices = similarities.argsort()[-3:][::-1]
    top_scores = similarities[top_indices]

    # Use threshold for relevance
    if top_scores[0] > 0.3:  # Stricter threshold
        # Randomly select from top 3 for variety
        chosen_idx = random.choice(top_indices[:2] if top_scores[1] > 0.3 else [top_indices[0]])
        character = data.iloc[chosen_idx]['name']
        response = data.iloc[chosen_idx]['line']
        return f"{character}: {response}"
    return random.choice([
        "Rick: burp Morty, your input’s dumber than a Gazorpazorpian!",
        "Morty: Aw, geez, I-I don’t know what you mean!",
        "Summer: Ugh, like, totally irrelevant, okay?"
    ])

try:
    # Load and preprocess data
    data = load_data()
    data = data[['name', 'line']].dropna()
    data['processed_line'] = data['line'].apply(preprocess_text)

    # Initialize TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    tfidf_matrix = vectorizer.fit_transform(data['processed_line'])

    # Interactive chatbot loop
    print("Rick and Morty Chatbot! Enter a message (e.g., 'I need an adventure'). Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Rick: Alright, Morty, let’s portal outta this boring dimension!")
            break
        response = get_response(user_input, data, vectorizer, tfidf_matrix)
        print(response)

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

Rick and Morty Chatbot! Enter a message (e.g., 'I need an adventure'). Type 'exit' to quit.
You: hi
Jessica: Hi, Morty.
You: i  need an adventure jessica
Morty: Jessica.
You: i need an adventure
Morty: Rick, since it's my adventure and all, could you do me a favor?
You: what kind of favour
Jerry: The "yes" or "no" kind.
You: yes
Pickle Rick: Yes.
You: exit
Rick: Alright, Morty, let’s portal outta this boring dimension!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]